In [19]:
import pandas as pd

df = pd.read_csv("SalesKaggle3.csv")
df.head()

,Order,File_Type,SKU_number,SoldFlag,SoldCount,MarketingType,ReleaseNumber,New_Release_Flag,StrengthFactor,PriceReg,ReleaseYear,ItemCount,LowUserPrice,LowNetPrice
0,2,Historical,1737127,0.0,0.0,D,15,1,682743.0,44.99,2015,8,28.97,31.84
1,3,Historical,3255963,0.0,0.0,D,7,1,1016014.0,24.81,2005,39,0.00,15.54
2,4,Historical,612701,0.0,0.0,D,0,0,340464.0,46.00,2013,34,30.19,27.97
3,6,Historical,115883,1.0,1.0,D,4,1,334011.0,100.00,2006,20,133.93,83.15
4,7,Historical,863939,1.0,1.0,D,2,1,1287938.0,121.95,2010,28,4.00,23.99


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198917 entries, 0 to 198916
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Order             198917 non-null  int64  
 1   File_Type         198917 non-null  object 
 2   SKU_number        198917 non-null  int64  
 3   SoldFlag          75996 non-null   float64
 4   SoldCount         75996 non-null   float64
 5   MarketingType     198917 non-null  object 
 6   ReleaseNumber     198917 non-null  int64  
 7   New_Release_Flag  198917 non-null  int64  
 8   StrengthFactor    198917 non-null  float64
 9   PriceReg          198917 non-null  float64
 10  ReleaseYear       198917 non-null  int64  
 11  ItemCount         198917 non-null  int64  
 12  LowUserPrice      198917 non-null  float64
 13  LowNetPrice       198917 non-null  float64
dtypes: float64(6), int64(6), object(2)
memory usage: 21.2+ MB


In [21]:
print(df["SoldFlag"].value_counts() / len(df["SoldFlag"]))

0.0    0.316715
1.0    0.065334
Name: SoldFlag, dtype: float64


In [22]:
# 재고 가치
df = df[df["PriceReg"] > 0]
df["stockvalue"] = df["ItemCount"] * df["LowNetPrice"]

In [23]:
# 재고 보관 기간
df["stockage"] = 2019 - df["ReleaseYear"]

In [24]:
df['discount_rate'] = 1.0 - (df['LowUserPrice'] / df['PriceReg'])
import numpy as np
df['discount_rate'] = df['discount_rate'].replace([-np.inf, np.nan], 0)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196368 entries, 0 to 198916
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Order             196368 non-null  int64  
 1   File_Type         196368 non-null  object 
 2   SKU_number        196368 non-null  int64  
 3   SoldFlag          74601 non-null   float64
 4   SoldCount         74601 non-null   float64
 5   MarketingType     196368 non-null  object 
 6   ReleaseNumber     196368 non-null  int64  
 7   New_Release_Flag  196368 non-null  int64  
 8   StrengthFactor    196368 non-null  float64
 9   PriceReg          196368 non-null  float64
 10  ReleaseYear       196368 non-null  int64  
 11  ItemCount         196368 non-null  int64  
 12  LowUserPrice      196368 non-null  float64
 13  LowNetPrice       196368 non-null  float64
 14  stockvalue        196368 non-null  float64
 15  stockage          196368 non-null  int64  
 16  discount_rate     19

In [26]:
df = df[df["File_Type"] == "Historical"]

In [27]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for column in df.select_dtypes(include=['object']).columns:
    df[column] = le.fit_transform(df[column])

df.head()

,Order,File_Type,SKU_number,SoldFlag,SoldCount,MarketingType,ReleaseNumber,New_Release_Flag,StrengthFactor,PriceReg,ReleaseYear,ItemCount,LowUserPrice,LowNetPrice,stockvalue,stockage,discount_rate
0,2,0,1737127,0.0,0.0,0,15,1,682743.0,44.99,2015,8,28.97,31.84,254.72,4,0.356079
1,3,0,3255963,0.0,0.0,0,7,1,1016014.0,24.81,2005,39,0.00,15.54,606.06,14,1.000000
2,4,0,612701,0.0,0.0,0,0,0,340464.0,46.00,2013,34,30.19,27.97,950.98,6,0.343696
3,6,0,115883,1.0,1.0,0,4,1,334011.0,100.00,2006,20,133.93,83.15,1663.00,13,-0.339300
4,7,0,863939,1.0,1.0,0,2,1,1287938.0,121.95,2010,28,4.00,23.99,671.72,9,0.967200


In [28]:
print(df["SoldFlag"].value_counts() / len(df["SoldFlag"]))

0.0    0.828917
1.0    0.171083
Name: SoldFlag, dtype: float64


In [29]:
X = df.drop(columns=["SoldFlag","SoldCount","Order","SKU_number","File_Type",
                    "ReleaseYear"])
y = df["SoldFlag"]

In [30]:
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

In [31]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

In [32]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X = scaler.fit_transform(X)

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [34]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(86573, 11)
(37103, 11)
(86573,)
(37103,)


## Random Forest

In [46]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators= 70, max_features = 'log2')
model.fit(X_train, y_train)

RandomForestClassifier(max_features='log2', n_estimators=80)

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, f1_score, fbeta_score

param_grid = {
    'n_estimators': [70],
    'max_features': ['log2'],
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)

Best parameters: {'max_features': 'log2', 'n_estimators': 80}


In [47]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"accuracy: {accuracy}")
print("\nconfusion matrix:")
print(confusion)
print("\nreport:")
print(report)
print("피처 중요도 : \n", model.feature_importances_)

accuracy: 0.8245424898256206

confusion matrix:
[[15139  3543]
 [ 2967 15454]]

report:
              precision    recall  f1-score   support

         0.0       0.84      0.81      0.82     18682
         1.0       0.81      0.84      0.83     18421

    accuracy                           0.82     37103
   macro avg       0.82      0.82      0.82     37103
weighted avg       0.82      0.82      0.82     37103

피처 중요도 : 
 [0.16902875 0.057746   0.01994426 0.13003676 0.09268868 0.11715106
 0.10081327 0.08438664 0.09085504 0.05624339 0.08110615]


In [194]:
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print(feature_importance)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

## XGBoost

In [63]:
from xgboost import XGBClassifier

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [400],
    'learning_rate': [0.1],
    'max_depth': [12],
    'subsample': [0.8],
    'colsample_bytree': [1],
    'gamma': [0, 0.01]
}

# GridSearchCV 설정
grid_search = GridSearchCV(estimator=XGBClassifier(random_state=42), param_grid=param_grid, verbose = 2, cv=5, n_jobs=-1, scoring='accuracy')

# 모델 학습
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best parameters:", grid_search.best_params_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters: {'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 12, 'n_estimators': 400, 'subsample': 0.8}


In [68]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=400, subsample=0.8, max_depth=15, learning_rate=0.1, gamma=0.01,
                      colsample_bytree=1)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              device=None, early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.01,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [69]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"accuracy: {accuracy}")
print("\nconfusion matrix:")
print(confusion)
print("\nreport:")
print(report)
print("피처 중요도 : \n", model.feature_importances_)

accuracy: 0.8406867369215427

confusion matrix:
[[15671  3011]
 [ 2900 15521]]

report:
              precision    recall  f1-score   support

         0.0       0.84      0.84      0.84     18682
         1.0       0.84      0.84      0.84     18421

    accuracy                           0.84     37103
   macro avg       0.84      0.84      0.84     37103
weighted avg       0.84      0.84      0.84     37103

피처 중요도 : 
 [0.73672825 0.02646989 0.07575829 0.02108518 0.01960712 0.02846835
 0.01898278 0.01878626 0.01771564 0.01764919 0.01874902]


In [173]:
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print(feature_importance)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

## KNN Classifier

In [72]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)

KNeighborsClassifier()

In [73]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"accuracy: {accuracy}")
print("\nconfusion matrix:")
print(confusion)
print("\nreport:")
print(report)

accuracy: 0.7624720373015659

confusion matrix:
[[13771  4911]
 [ 3902 14519]]

report:
              precision    recall  f1-score   support

         0.0       0.78      0.74      0.76     18682
         1.0       0.75      0.79      0.77     18421

    accuracy                           0.76     37103
   macro avg       0.76      0.76      0.76     37103
weighted avg       0.76      0.76      0.76     37103



## Logistic Regression

In [198]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

LogisticRegression(max_iter=200)

In [199]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"accuracy: {accuracy}")
print("\nconfusion matrix:")
print(confusion)
print("\nreport:")
print(report)

accuracy: 0.7566931216931216

confusion matrix:
[[13343  5617]
 [ 3580 15260]]

report:
              precision    recall  f1-score   support

         0.0       0.79      0.70      0.74     18960
         1.0       0.73      0.81      0.77     18840

    accuracy                           0.76     37800
   macro avg       0.76      0.76      0.76     37800
weighted avg       0.76      0.76      0.76     37800



In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# X_train = pd.DataFrame(X_train, columns = X.columns)

In [200]:
import statsmodels.api as sm

logreg = sm.Logit(y_train, sm.add_constant(X_train)).fit()
print(logreg.summary())

Optimization terminated successfully.
         Current function value: 0.517769
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:               SoldFlag   No. Observations:                88200
Model:                          Logit   Df Residuals:                    88188
Method:                           MLE   Df Model:                           11
Date:                Wed, 11 Dec 2024   Pseudo R-squ.:                  0.2530
Time:                        09:14:29   Log-Likelihood:                -45667.
converged:                       True   LL-Null:                       -61135.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.8728      5.891     -0.657      0.511     -15.420       7.674
x1            -2.1613      0.

In [ ]:
import numpy as np

odds = np.exp(logreg.params)

for i in range(1, len(odds)):
    print(f'변수 {X_train.columns[i-1]}의 logit : {logreg.params[i] : .3f}')
    print(f'변수 {X_train.columns[i-1]}가 1단위 증가할 때, 악성재고일 로짓(odds ratio에 자연로그를 취한 값)이{logreg.params[i] : .3f}배 증가한다.')
    print(f'변수 {X_train.columns[i-1]}의 odds ratio : {odds[i] : .3f}')
    print(f'변수 {X_train.columns[i-1]}가 1단위 증가할 때, 악성재고일 확률(종속변수가 1일 확률)이 그렇지 않을 경우보다{odds[i] : .3f}배 증가한다.\n')

In [ ]:
sig_level = .05
logistic_variable = pd.concat([logreg.params, np.exp(logreg.params), np.round(logreg.pvalues,4)], axis=1)
logistic_variable.columns = ['logit', 'odds ratio', 'p-value']
logistic_variable.loc[logistic_variable['p-value'] > sig_level, f'above {sig_level*100}%'] = 'No'
logistic_variable.loc[logistic_variable['p-value'] < sig_level, f'above {sig_level*100}%'] = 'Yes'
print(logistic_variable)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

cm_df = pd.DataFrame(logreg.pred_table())
cm_df.columns = ['Predicted Active', 'Predicted Historical']
cm_df = cm_df.rename(index={0: 'Actual Active',1: 'Actual Historical'})
print(cm_df)

sns.heatmap(cm_df, cmap = 'RdYlBu_r', annot = True, annot_kws={"size": 10}, linewidths=.5, fmt='.0f')
plt.title('confusion matrix')
plt.xlabel('actual')
plt.ylabel('pred')
plt.show()

## Decision Tree

In [201]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(criterion='gini', max_depth=7,
                               min_samples_leaf=2, min_samples_split=2,
                               max_features=12)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=7, max_features=12, min_samples_leaf=2)

In [202]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"accuracy: {accuracy}")
print("\nconfusion matrix:")
print(confusion)
print("\nreport:")
print(report)

accuracy: 0.768095238095238

confusion matrix:
[[13967  4993]
 [ 3773 15067]]

report:
              precision    recall  f1-score   support

         0.0       0.79      0.74      0.76     18960
         1.0       0.75      0.80      0.77     18840

    accuracy                           0.77     37800
   macro avg       0.77      0.77      0.77     37800
weighted avg       0.77      0.77      0.77     37800



## SVC

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='linear', C= 1, gamma=1, class_weight=None, max_iter=100,
            random_state=1202)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"accuracy: {accuracy}")
print("\nconfusion matrix:")
print(confusion)
print("\nreport:")
print(report)

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"accuracy: {accuracy}")
print("\nconfusion matrix:")
print(confusion)
print("\nreport:")
print(report)

## Hist Gradient Classifier

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

model = HistGradientBoostingClassifier(loss='log_loss', learning_rate=0.1, max_iter=1000, max_leaf_nodes=31, max_depth=7,
                                       min_samples_leaf=10)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"accuracy: {accuracy}")
print("\nconfusion matrix:")
print(confusion)
print("\nreport:")
print(report)

## Stacking Classifier

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.ensemble import StackingClassifier

estimators = [
    ('rf', RandomForestClassifier(n_estimators=300)),
    ('svr', make_pipeline(StandardScaler(),LinearSVC()))]

In [ ]:
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
clf.fit(X_train, y_train).score(X_test, y_test)

## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(n_estimators=500, algorithm="SAMME",learning_rate=1.3)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"accuracy: {accuracy}")
print("\nconfusion matrix:")
print(confusion)
print("\nreport:")
print(report)

## Ridge Classifier

In [ ]:
from sklearn.linear_model import RidgeClassifierCV

model = RidgeClassifierCV(cv=10)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"accuracy: {accuracy}")
print("\nconfusion matrix:")
print(confusion)
print("\nreport:")
print(report)